In [9]:
import requests
import pandas as pd

# Define the Overpass API endpoint
overpass_url = "http://overpass-api.de/api/interpreter"

# Create the query to fetch all cities in Great Britain
overpass_query = """
[out:json][timeout:1800];
area["name"="Great Britain"]->.searchArea;
(
  node["place"="city"](area.searchArea);
  way["place"="city"](area.searchArea);
  relation["place"="city"](area.searchArea);
);
out body;
>;
out skel qt;
"""

# Send the query to the Overpass API
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# Extract relevant data
elements = data['elements']
cities = []
for element in elements:
    if 'tags' in element and 'name' in element['tags']:
        city = {
            'id': element['id'],
            'name': element['tags']['name'],
            'type': element['type'],
            'lat': element.get('lat', None),
            'lon': element.get('lon', None),
            'nodes': element.get('nodes', None),
            'members': element.get('members', None)
        }
        cities.append(city)

# Convert to DataFrame
cities_df = pd.DataFrame(cities)
# print(cities_df)

In [10]:
cities_df

,id,name,type,lat,lon,nodes,members
0,107775,London,node,51.507446,-0.127765,None,None
1,304611,Wells,node,51.209451,-2.645120,None,None
2,358309,Leeds,node,53.797418,-1.543794,None,None
3,1947201,Bath,node,51.381386,-2.359696,None,None
4,8087537,Chester,node,53.190887,-2.890896,None,None
...,...,...,...,...,...,...,...
76,5342409,Bath,relation,NaN,NaN,None,"[{'type': 'node', 'ref': 1947201, 'role': 'adm..."
77,5746665,Bristol,relation,NaN,NaN,None,"[{'type': 'way', 'ref': 484411949, 'role': 'ou..."
78,13948853,St Albans,relation,NaN,NaN,None,"[{'type': 'node', 'ref': 25342557, 'role': 'la..."
79,14164364,Dunfermline,relation,NaN,NaN,None,"[{'type': 'way', 'ref': 1062172740, 'role': 'o..."


In [ ]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('main_db.sqlite')
cursor = conn.cursor()

# Create the osm_cities table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS osm_cities (
    id INTEGER PRIMARY KEY,
    name TEXT,
    type TEXT,
    lat REAL,
    lon REAL,
    nodes TEXT,
    members TEXT
)
''')

# Insert the data into the osm_cities table
for _, row in cities_df.iterrows():
    cursor.execute('''
    INSERT OR REPLACE INTO osm_cities (id, name, type, lat, lon, nodes, members)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (row['id'], row['name'], row['type'], row['lat'], row['lon'], str(row['nodes']), str(row['members'])))

# Commit the transaction and close the connection
conn.commit()
conn.close()